Comare and evaluate how our different classifiers have done based on the test data not used for training.

In [ ]:
import pymongo
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from extcats import CatalogQuery
from scipy.stats import binned_statistic

In [ ]:
client = pymongo.MongoClient()

In [ ]:
db = client.TrainingValidation

In [ ]:
db.list_collection_names()

In [ ]:
col = db.t2

Extract output from T2XgbClassifier

In [ ]:
xgbdata = []

In [ ]:
for t2info in col.find({'unit':'T2XgbClassifier', "code":0}):
    b = t2info['body'][-1]
    b['stock'] = t2info['stock']
    b['config'] = t2info['config']
    b['code'] = t2info['code']
    b['model'] = t2info['channel'][0]
    b['link'] = t2info['link']
    xgbdata.append( b )

In [ ]:
df_xgb = pd.DataFrame.from_dict(xgbdata)

In [ ]:
df_xgb.shape

In [ ]:
tabdata = []

In [ ]:
for t2info in col.find({'unit':'T2TabulatorRiseDecline', "code":0}):
    # Only store required info
    #b = t2info['body'][-1]
    b = {}
    b['stock'] = t2info['stock']
    b['link'] = t2info['link']
    b['ndet'] = t2info['body'][-1]['ndet']
    b['success'] = t2info['body'][-1]['success']
    tabdata.append( b )

In [ ]:
df_tab = pd.DataFrame.from_dict(tabdata)

In [ ]:
df_tab.shape

Why did we retrieve the Tabulator data? Since the xgbdata output does not store (well) how many detections was used to construct the guess. In the end we will want to study how the classifications work as a function of detections. So we would wish to complement the xgbdata with the number of detections.

In [ ]:
df_xgbtab = None

In [ ]:
df_xgbtab = pd.merge(df_xgb, df_tab, on='link', how='left')

In [ ]:
df_xgbtab['prob0']

In [ ]:
plt.hist(df_xgbtab['ndet'], bins=30)

In [ ]:
set(df_xgbtab['config'])

How do we evaluate this? For a particular model (i.e. agn) and config we can define what the true result should be. So steps:
- Figure out which config corresponds to which model. 
- For each of the configs, write a list of the models with which `is_0` is the True outcome. 
- Make some nice plots... We can play around with ndet, models, configs and prob0 so probably need to create some plots. 

From confid collection in TrainingValidation:
5560207956784798794 xgb_v6_tree12_
2508649458706045692 xgb_v6_tree121113_ 
-5649973164721472516 xgb_v6_tree2122_ 
8559538834424564852 xgb_v6_simmod_tree12_ 
2616828516668705481 xgb_v6_simmod_tree2122_ 
-6030627619870677701 xgb_v6_simmod_tree121113_
As I recall their definition the first item would be the target for "0". 

In [ ]:
taxonomy = {
    11:[
        'sniasalt2',
 'snictemplates',
 'snibhostxtv19',
 'snibtemplates',
 'snichostxtv19',
 'snicblhostxtv19',
 'sniitemplates',
 'sniibhostxtv19',
 'sniinmosfit',
 'sniihostxtv19',
 'sniinmf',
 'sniinhostxtv19',
 'sniax',
 'snia91bg',
    ],
 'snia':[
    'sniasalt2',
    'sniax',
     'snia91bg',
    ],
  'sni':[
        'sniasalt2',
 'snictemplates',
 'snibhostxtv19',
 'snibtemplates',
 'snichostxtv19',
 'snicblhostxtv19',
 'sniax',
 'snia91bg',
    ],
  'snibc':[
 'snictemplates',
 'snibhostxtv19',
 'snibtemplates',
 'snichostxtv19',
 'snicblhostxtv19',
    ],
  'snii':[
 'sniitemplates',
 'sniibhostxtv19',
 'sniinmosfit',
 'sniihostxtv19',
 'sniinmf',
 'sniinhostxtv19',
    ],
    12:[
 'knk17',
 'knb19',
 'mdwarfflare',
 'dwarfnova',
 'ulenssinglegenlens',
 'ulenssinglepylima',
 'ulensbinary',            
    ],
    'ulens':[
 'ulenssinglegenlens',
 'ulenssinglepylima',
 'ulensbinary',            
    ],
    'kn':[
 'knk17',
 'knb19',
    ],    
    13:[
 'slsnihost',
 'slsninohost',
 'tde',
 'ilot',
 'cart',
 'pisn',            
    ],
    'slsni':[
 'slsnihost',
 'slsninohost',
    ],
    21:[
 'cepheid',
 'rrl'
 'dsct',
 'eb',          
    ],
    22:[ 'agn',]           
           }

In [ ]:
modelmaps = {}

# xgb_v6_tree12_
modelmaps[5560207956784798794] = {
    'is0':[ 'sniasalt2',
 'snictemplates',
 'snibhostxtv19',
 'snibtemplates',
 'snichostxtv19',
 'snicblhostxtv19',
 'sniitemplates',
 'sniibhostxtv19',
 'sniinmosfit',
 'sniihostxtv19',
 'sniinmf',
 'sniinhostxtv19',
 'sniax',
 'snia91bg',
 'knk17',
 'knb19',
 'mdwarfflare',
 'dwarfnova',
 'ulenssinglegenlens',
 'ulenssinglepylima',
 'ulensbinary',
 'slsnihost',
 'slsninohost',
 'tde',
 'ilot',
 'cart',
 'pisn',],
    'is1':[
         'cepheid',
 'rrl'
 'dsct',
 'eb',
 'agn',
]
    
    
}

# xgb_v6_tree121113_
modelmaps[2508649458706045692] = {
    'is0': [ 'knk17',
 'knb19',
 'mdwarfflare',
 'dwarfnova',
 'ulenssinglegenlens',
 'ulenssinglepylima',
 'ulensbinary',],
    'is1': [
 'sniasalt2',
 'snictemplates',
 'snibhostxtv19',
 'snibtemplates',
 'snichostxtv19',
 'snicblhostxtv19',
 'sniitemplates',
 'sniibhostxtv19',
 'sniinmosfit',
 'sniihostxtv19',
 'sniinmf',
 'sniinhostxtv19',
 'sniax',
 'snia91bg', 
 'knk17',
 'knb19',
 'mdwarfflare',
 'dwarfnova',
 'ulenssinglegenlens',
 'ulenssinglepylima',
 'ulensbinary',        
    ],
}

In [ ]:
# xgb_v6_tree2122_ 
modelmaps[-5649973164721472516] = {
    'is0':[
        'cepheid',
 'rrl'
 'dsct',
 'eb',
    ],
    'is1': ['agn']
}

In [ ]:
# xgb_v6_tree12_
modelmaps[5560207956784798794] = {
    'is0':[ 11, 12, 13],
    'is1':[ 21, 22]
}
# xgb_v6_tree121113_
modelmaps[2508649458706045692] = {
    'is0':[ 12],
    'is1':[ 11, 13]
}
# xgb_v6_tree2122_ 
modelmaps[-5649973164721472516] = {
    'is0':[ 21],
    'is1':[ 22]
}    

In [ ]:
# The three others are variants of the first
# xgb_v6_simmod_tree12_
modelmaps[8559538834424564852] = modelmaps[5560207956784798794] 
# 2616828516668705481 xgb_v6_simmod_tree2122_
modelmaps[2616828516668705481] = modelmaps[-5649973164721472516]
# -6030627619870677701 xgb_v6_simmod_tree121113_
#modelmaps[-6030627619870677701] = modelmaps[2508649458706045692]
# or is this not reversed?
modelmaps[-6030627619870677701] = {
    'is0':[ 11, 13],
    'is1':[ 12]
}

In [ ]:
modelmaps

In [ ]:
# This is what we need

def get_modellist(taxonomies):
    '''
    For a list of taxonomies, return a list of the models which should be included.
    If the list is instead models, these will just be propagated.
    '''
    models = []
    for tax in taxonomies:
        if tax in taxonomy.keys():            
            models.extend( taxonomy[tax] )
        else:
            models.append(tax)
    return models
 

def get_modelindex(models, mydf=df_xgbtab):
    '''
    For a list of models, return an index of rows correspondings to any of these. 
    '''
    return mydf['model'].isin(models)
    
    
def get_classifier_results(config, mydf=df_xgbtab, modelmaps=modelmaps):
    '''
    For a particular config, return a tuple
    (ndet, ptrue, is1type)
    where ndet is the number of detections and ptrue is the (possibly 
    inverted) provided probability that would have been correct. An ideal
    classifier would thus have consistent one.
    '''
    
    # 1. Find the subset relevant for this config
    df_conf = mydf[mydf['config']==config]

    # 2. Find rows which corresponds to H0 and H1 for this particular classifier.
    iH0 = get_modelindex( get_modellist(modelmaps[config]['is0']), mydf=df_conf)
    iH1 = get_modelindex( get_modellist(modelmaps[config]['is1']), mydf=df_conf)
    
    # 3. Process the H0 results
    ndet_h0 = df_conf['ndet'][iH0]
    # So this is the probability to be H0, which for these objects would be true.
    # So prob0==correct
    success_h0 = df_conf['prob0'][iH0]
    type_h0 = np.zeros(len(success_h0))

    # 4. Process the H1 results
    ndet_h1 = df_conf['ndet'][iH1]
    # So this is the probability to be H0, which for these objects would be false.
    # So (1-prob0)==correct
    success_h1 = 1.-df_conf['prob0'][iH1]
    type_h1 = np.ones(len(success_h1))
    
    # 5. Return     
    return pd.concat([ndet_h0,ndet_h1]), pd.concat([success_h0,success_h1]), np.append( type_h0, type_h1)


def get_model_results(config, model, mydf=df_xgbtab, modelmaps=modelmaps):
    '''
    For a particular config and model, return a tuple
    (ndet, ptrue)
    where ndet is the number of detections and ptrue is the (possibly 
    inverted) provided probability that would have been correct. 
    '''
    
    # 1. Find the subset relevant for this config
    df_conf = mydf[mydf['config']==config]

    # 2. Find whether this model corresponds to the H0 or H1 predictions (or none)
    if model in get_modellist(modelmaps[config]['is0']):
        print('H0 model')
        h1 = False
    elif model in get_modellist(modelmaps[config]['is1']):
        print('H1 model')
        h1 = True
    else:
        print('Classifier not trained for this model')
        return None, None
    # Get model inidces
    iModel = get_modelindex( [model], mydf=df_conf)
    
    
    # 4. Process the H1 results
    ndet = df_conf['ndet'][iModel]
    # So this is the probability to be H0, which for these objects would be false.
    # So (1-prob0)==correct
    if h1:
        success = 1.-df_conf['prob0'][iModel]
    else:
        success = df_conf['prob0'][iModel]
    
    # 5. Return     
    return ndet, success    



In [ ]:
det, suc = get_model_results(2508649458706045692, 'knk17')

In [ ]:
det, suc, typ  = get_classifier_results(8559538834424564852)

In [ ]:
plt.plot(det, suc, 'o')

In [ ]:
# Fixed ndet cuts 
ndet_cuts = [-1.0e-03,  1.0e+00,  2.0e+00,  4.0e+00,  5.0e+00,
        8.0e+00,  1.1e+01, 1.6e+01,  1.9e+01,  2.2e+01,  2.6e+01,
        3.0e+01,  3.6e+01,  4.5e+01,  6.3e+01, 10**3]

In [ ]:
#fep = pd.qcut(det, 30, duplicates='drop')
fep = pd.cut(det, ndet_cuts)

In [ ]:
suc2

In [ ]:
suc2 = pd.DataFrame({'suc':suc, 'qdet':fep, 'ndet':det})

In [ ]:
confmean = suc2.groupby(['qdet']).mean()
confstd = suc2.groupby(['qdet']).std()
confcount = suc2.groupby(['qdet']).count()


In [ ]:
confmean

In [ ]:
suc2['qdet'].values

In [ ]:
ndet_bins = np.unique([ (v.right-v.left)/2+v.left for v in suc2['qdet'].values] ) 

In [ ]:
plt.errorbar(confmean['ndet'], confmean['suc'], yerr=confstd['suc']/np.sqrt(confcount['suc']), fmt='.')
plt.plot(confmean['ndet'], confmean['suc'], 'o', ms=12, color='dodgerblue')
plt.xlabel('Number of (significant) detection')
plt.ylabel('Prob of classification being correct')
plt.savefig('/home/jnordin/tmp/modeleval.png')

With the above we can study some things:
- For each individual model, how well do the different classifiers work (as a function of number of detectoins).
- For each taxonomy group, how well do the classifiers work?
- Can we improve things through changing e.g. range of detections?

Next sep:
- Make some nice summary plot that in one figure compares how the different classifiers work.
- For each classifier, draw lines for all of the different models to study whether some work particularly bad. 
- Look at parsnip, possible under the constraint of using the fine-grained tree.

In [ ]:
for config, group in df_xgb.groupby(['model','config']):
    print('Model {}'.format(group['model'].iloc[0]))
    print('Config {} for {} fits out of which {} failed.'.format(config, group.shape[0], sum(group['xgbsuccess']==False)))
    sub = group[ ~(group['xgbsuccess']==False) ]
    print('Out of the remaining {}, {} are classified as class O ({})'.format(
                        sub.shape[0], sum(group['is_0']==True), sum(group['is_0']==True)/sub.shape[0]))
    plt.figure()
#    plt.title(sub['model'][0])
    plt.hist(sub['imodel'])
    plt.figure()
#    plt.title(sub['model'][0])
    plt.hist(sub['prob0'])

    

In [ ]:
parsnipdata = []

In [ ]:
# When we are evaluating the different t2runparsnip runs we have
# 2244063382525234836   -     old config (w/o ulens)
# 9048316057906136269   -     new config (w/o SLSN)

In [ ]:
for t2info in col.find({'unit':'T2RunParsnip', "code":0}):
    try:
        classes = t2info['body'][0]['classification']
        classes['dof'] = t2info['body'][0]['prediction']['model_dof']
        classes['chisq'] = t2info['body'][0]['prediction']['model_chisq']
    except KeyError:
        classes = {}
    classes['model'] = t2info['channel'][0]
    classes['stock'] = t2info['stock']
    classes['link'] = t2info['link']    
    classes['config'] = t2info['config']    
    parsnipdata.append( classes )

In [ ]:
df_parsnip = pd.DataFrame.from_dict(parsnipdata)

In [ ]:
# Update that column names to match our taxonomy
parsnip_colmap = {'CART':'cart', 'ILOT':'ilot', 'KN':'kn', 'Mdwarf-flare':'mdwarfflare', 
                  'PISN':'pisn', 'SLSN-I':'slsni', 'SNII':'snii', 'SNIa':'sniasalt2',
             'SNIa91bg':'snia91bg', 'SNIax':'sniax', 'SNibc':'snibc', 'TDE':'tde', 
                  'dwarf-nova':'dwarfnova', 'uLens':'ulens'
}

In [ ]:
df_parsnip.columns

In [ ]:
df_parsnip['model'].unique()

In [ ]:
#for oname, newname in parsnip_colmap.items():
df_parsnip.rename(parsnip_colmap, axis=1, inplace=1)

How do we wish to evaluate parsnip? Here we have more moving parts in terms of specific models, dof and chi2, as well as whether we consider different subsets. 
So we need to take into account:
- That some taxonomy groups could have been counted as removed (or not).
- That we could have some limits to dof and chi.
- Plot both the fraction of correctly classified objects as well as well as other kind of objects classified as this. 

When we have two separate parsnip runs, who should these be evaluated? For first stage it is probably easiest to create two files and then switch which df_parsnip points to.

In [ ]:
df_parsnipm1 = df_parsnip[df_parsnip['config']==2244063382525234836]
df_parsnipm2 = df_parsnip[df_parsnip['config']==9048316057906136269]

In [ ]:
# Simply reimplement these for the parsnip datafiles?

 

def parsnip_modelcolumns(models, mydf=df_parsnip):
    '''
    For a parsnip datafile, return the subset of columns correspond to the listed models.
    Addid for completeness
    '''
    return mydf[models]
    

def parsnip_models_results(pred_models, true_models, marginalize_models = [], mydf=df_parsnip):
    '''
    For a list of model, return a tuple
    (ndet, chisq, pvalue, is_model)
    where ndet is the number of detections, chisq the fit chisq value 
    and pvalue to _summed_ probability values 
    (summed if results from multiple models are used)
    and is_model a boolean describing whether the fitted model is the expected.
    
    marginalize_models correspond to a list of column (predicted models) which should 
    first be removed and the remaining probabilities rescaled.
    '''
    
    # 0. Marginalize
    if len(marginalize_models)>0:
        df_marge = parsnip_modelcolumns(marginalize_models, mydf=mydf)
        marge_prob = 1. / ( 1.-df_marge.sum(axis=1) )
    else:
        marge_prob = 1.
    
    # 1. Get the probabilities 
    # Here we should not use the taxonomy expansion, we are assuming that 
    # we are using columns from parsnip_colmap
    df_pred = parsnip_modelcolumns( pred_models, mydf=mydf)    
    prob = df_pred.sum(axis=1)
    prob *= marge_prob

    # 2. Find which rows were generated based on the list of models.
    iModel = get_modelindex( get_modellist(true_models), mydf=mydf)
    is_model = np.zeros(len(prob),dtype=bool)
    is_model[iModel] = True
    
    return mydf['dof'], mydf['chisq'], prob, is_model
    
    
def parsnip_cut_models(cut_models, mydf=df_parsnip, modelmaps=modelmaps):
    '''
    Return a version of the datafiles where a number of models have been cut
    *and* the remaining model columns rescaled such that probabilities add up 
    to one.
    '''
    
    # 1. Get the probabilities of the cut columns
    df_pred = parsnip_modelcolumns(pred_models, mydf=mydf)
    cut_prob = df_pred.sum(axis=1)
    
    # 1. Find the subset relevant for this config
    df_conf = mydf[mydf['config']==config]

    # 2. Find rows which corresponds to H0 and H1 for this particular classifier.
    iH0 = get_modelindex( get_modellist(modelmaps[config]['is0']), mydf=df_conf)
    iH1 = get_modelindex( get_modellist(modelmaps[config]['is1']), mydf=df_conf)
    
    # 3. Process the H0 results
    ndet_h0 = df_conf['ndet'][iH0]
    # So this is the probability to be H0, which for these objects would be true.
    # So prob0==correct
    success_h0 = df_conf['prob0'][iH0]
    type_h0 = np.zeros(len(success_h0))

    # 4. Process the H1 results
    ndet_h1 = df_conf['ndet'][iH1]
    # So this is the probability to be H0, which for these objects would be false.
    # So (1-prob0)==correct
    success_h1 = 1.-df_conf['prob0'][iH1]
    type_h1 = np.ones(len(success_h1))
    
    # 5. Return     
    return pd.concat([ndet_h0,ndet_h1]), pd.concat([success_h0,success_h1]), np.append( type_h0, type_h1)





In [ ]:
taxonomy

In [ ]:
dof, chisq, prob, is_model = parsnip_models_results(['ulens'], ['ulens'], mydf=df_parsnipm2)

In [ ]:
plt.plot(dof[is_model], prob[is_model], 'o')

Time to return to the question of what it is we want to find out.
- Is parsnip working good enough, i.e. at all as good as we expect?
- Would it be meaningfully improved by limiting to some taxonomies (i.e. trusting xgb further)
- Would it improve to include some min requirements on dof and/or chisq/dof?
- How much of the "bad" behaviour is caused by similar models being wrongly classified, i.e. things you could accept?

The basic plot we would like to see is 

In [ ]:
# Cuts for plot
min_dof = 0
min_chisqdof = 0
max_chisqdof = 99



In [ ]:
def get_parsnip_cutbins(dof, chisq, prob, 
                        min_dof=0, min_chisqdof=0, max_chisqdof=999, xcol='dof',bins=10):
    """
    Return version of data results which have been cut and then binned.
    
    Bins can either be a number of bins, or a spcific set of bins.
    """
    
    # Do cuts
    chisqdof = chisq/dof
    iCut = ( (dof>=min_dof) & (chisqdof>=min_chisqdof)  & (chisqdof<=max_chisqdof))
    
    if xcol=='dof':
        xval = dof
    elif xcol=='chisqdof':
        xval = chisqdof
    
    # Binning
    if isinstance(bins, int):
        bingroup = pd.qcut(xval[iCut], bins, duplicates='drop')
    else:
        bringroup = pd.cut(xval[iCut], bins)
    
    # Bin and do simple stats
    fp_tmp = pd.DataFrame({'prob':prob[iCut], 'ibin':bingroup, 'xval':xval[iCut]})    
    confmean = fp_tmp.groupby(['ibin']).mean()
    confstd = fp_tmp.groupby(['ibin']).std()
    confcount = fp_tmp.groupby(['ibin']).count()
#    print(confmean)
    
    return confmean['xval'], confmean['prob'], confstd['prob']/np.sqrt(confcount['prob'])

In [ ]:
plt.figure(figsize=(8,6))
#plt.title(model)
    
for model in parsnip_colmap.values():
    
    # This is either a single model, or a taxonomy composite
    
    print(model)
#    models = get_modellist([model])
#    print('translated into models')

    # Check predictions of being this model
    dof, chisq, prob, is_model = parsnip_models_results([model], [model], mydf=df_parsnipm1)
    # The probability 
    # dof[is_model] ...
    
    # Bin according to number of detections and plot
    pos, prob, prob_err = get_parsnip_cutbins(dof[is_model], chisq[is_model], prob[is_model],
                                             xcol='chisqdof')
    
    plt.errorbar(pos, prob, yerr=prob_err, fmt='.', color='black')
#    plt.plot(pos,prob, 'o', ms=12, color='dodgerblue')
    plt.plot(pos,prob, 'o', ms=12, label=model)
plt.legend()
plt.xlabel('Chisq / d.o.f.')
plt.ylabel('Prob of classification being correct')
plt.xlim(0,5)
plt.show()
# Conclusion: there is no enormous improvement to justify cutting on d.o.f.

In [ ]:
plt.figure(figsize=(12,10))
#plt.title(model)
    
for model in parsnip_colmap.values():
    
    # This is either a single model, or a taxonomy composite
    
    print(model)
#    models = get_modellist([model])
#    print('translated into models')

    # Check predictions of being this model
    dof, chisq, prob, is_model = parsnip_models_results([model], [model], mydf=df_parsnipm2)
    # The probability 
    # dof[is_model] ...
    
    # Bin according to number of detections and plot
    pos, prob, prob_err = get_parsnip_cutbins(dof[is_model], chisq[is_model], prob[is_model],
                                             xcol='dof')
    
    plt.errorbar(pos, prob, yerr=prob_err, fmt='.', color='black')
#    plt.plot(pos,prob, 'o', ms=12, color='dodgerblue')
    plt.plot(pos,prob, 'o', ms=12, label=model)
plt.legend()
plt.xlabel('Number of degrees of freedom')
plt.ylabel('Prob of classification being correct')
#plt.xlim(0,20)
plt.show()
# Conclusion: Similarly, there is no gigantic improvement with more datapoint.
# it does get better, but not drasticly so.

In [ ]:
# What models should we marginalize over (remove)?
print(df_parsnip.columns)
print(set(df_parsnip['model']))
remove_models = ['kn', 'cart', 'ilot', 'mdwarfflare', 'dwarfnova', 'pisn', 'tde']

In [ ]:
# We now redo the above for the most interesting groups
taxgroups = ['snia', 'snii', 'snibc', 'slsni']
parsnip_taxnames = {'snii':['snii'], 'snibc':['snibc'], 'slsni':['slsni']}
for taxgroup in taxgroups:
    simmodels = get_modellist([taxgroup])
    print(simmodels)
    if taxgroup in parsnip_taxnames.keys():
        parsnip_models = parsnip_taxnames[taxgroup]
    else:
        parsnip_models = simmodels
    print(taxgroup, simmodels, parsnip_models)
    # Get results
    dof, chisq, prob, is_model = parsnip_models_results(parsnip_models, simmodels, 
                                                        remove_models, mydf=df_parsnipm2)
    # Bin according to number of detections and plot
    pos, prob, prob_err = get_parsnip_cutbins(dof[is_model], chisq[is_model], prob[is_model],
                                             xcol='dof')
    
    plt.errorbar(pos, prob, yerr=prob_err, fmt='.', color='black')
    plt.plot(pos,prob, 'o', ms=12, label=taxgroup)
plt.legend()
plt.xlabel('Number of degrees of freedom')
plt.ylabel('Prob of classification being correct')
#plt.xlim(0,20)
plt.show()

In [ ]:
# We now redo the above for the most interesting groups
taxgroups = ['snia', 'snii', 'snibc', 'slsni']
parsnip_taxnames = {'snii':['snii'], 'snibc':['snibc'], 'slsni':['slsni']}
for taxgroup in taxgroups:
    simmodels = get_modellist([taxgroup])
    print(simmodels)
    if taxgroup in parsnip_taxnames.keys():
        parsnip_models = parsnip_taxnames[taxgroup]
    else:
        parsnip_models = simmodels
    # Get results
    dof, chisq, prob, is_model = parsnip_models_results(parsnip_models, simmodels, 
                                                        mydf=df_parsnipm1)
    # Bin according to number of detections and plot
    pos, prob, prob_err = get_parsnip_cutbins(dof[is_model], chisq[is_model], prob[is_model],
                                             xcol='chisqdof')
    
    plt.errorbar(pos, prob, yerr=prob_err, fmt='.', color='black')
    plt.plot(pos,prob, 'o', ms=12, label=taxgroup)
plt.legend()
plt.xlabel('Chi / dof')
plt.ylabel('Prob of classification being correct')
plt.xlim(0,5)
plt.show()

In [ ]:
# Results are so bad, could we have mixed up columns? Look at some models
simmodel = 'ulens'
# prolematics:
# sniibhostxtv19
# slsninohost
# ulens*
pmodels = ['cart', 'ilot', 'kn', 'mdwarfflare', 'pisn', 'slsni', 'snii',
       'sniasalt2', 'snia91bg', 'sniax', 'snibc', 'tde', 'dwarfnova']
#pmod = [pmodels[12]]

plt.figure(figsize=(12,12))
for pmod in pmodels:
    dof, chisq, prob, is_model = parsnip_models_results([pmod], [simmodel], mydf=df_parsnipm2)
    # Bin according to number of detections and plot
    pos, prob, prob_err = get_parsnip_cutbins(dof[is_model], chisq[is_model], prob[is_model],
                                             xcol='dof')
    
    plt.errorbar(pos, prob, yerr=prob_err, fmt='.', color='black')
    plt.plot(pos,prob, 'o', ms=12, label=pmod)
plt.legend()
plt.title('Simulation of model '+simmodel)
plt.xlabel('Number of degrees of freedom')
plt.ylabel('Prob to be classified as')
#plt.xlim(0,20)
plt.show()

In [ ]:
# Which models should we look at?
#pmodels = ['snii', 'snibc', 'sniasalt2', 'snia91bg', 'sniax', 
#    'cart', 'ilot', 'pisn', 'slsni', 'tde', 'kn', 'mdwarfflare', 'dwarfnova']
#pmodels = ['snii', 'snibc', 'sniasalt2', 'snia91bg', 'sniax', 
#    'cart', 'ilot', 'pisn', 'slsni', 'tde']
#pmodels = ['snii', 'snibc', 'sniasalt2', 'snia91bg', 'sniax', ]
pmodels = ['kn', 'mdwarfflare', 'dwarfnova', 'ulens']


In [ ]:
df_parcut = df_parsnipm2.copy(deep=True)
#df_parcut = df_parsnipm2.copy(deep=True)
# Now, let us look at the most likely classification for each row
df_parcut['pmax'] = df_parsnipm2[pmodels].idxmax(axis=1)

In [ ]:
df_parcut['pmax']

In [ ]:
# Next question will be to map the simulation models to the pmax values
# these are the mappings we will need to do (the rest we can leave )
modmap = {'knb19':'kn', 'knk17':'kn', 'slsnihost':'slsni', 'slsninohost':'slsni', 
         'snibhostxtv19':'snibc', 'snibtemplates':'snibc', 'snicblhostxtv19':'snibc',
         'snichostxtv19':'snibc', 'snictemplates':'snibc', 'sniibhostxtv19':'snii',
         'sniihostxtv19':'snii', 'sniinhostxtv19':'snii', 'sniinmf': 'snii', 'sniinmosfit':'snii',
         'sniitemplates':'snii', 'ulenssinglegenlens':'ulens', 'ulenssinglepylima':'ulens', 
          'ulensbinary':'ulens'}
for mod in list(set(df_parcut['pmax'])):
    modmap[mod] = mod

# Remove some particularly bad models
#modmap.pop('slsninohost')
#modmap.pop('sniibhostxtv19')

print(modmap)

In [ ]:
df_parcut['model_parsnipconf'] = np.nan

In [ ]:
df_parcut['model_parsnipconf'] = df_parcut['model'].map(modmap)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
# Which model and in what order to use?

In [ ]:
# We either use all models that we included in pmodels, or all...
#iMatrix = (~df_parcut['model_parsnipconf'].isna()) & (~df_parcut['pmax'].isna()) & df_parcut['model_parsnipconf'].isin(pmodels)
iMatrix = (~df_parcut['model_parsnipconf'].isna()) & (~df_parcut['pmax'].isna())

In [ ]:
#labels = list( set(df_parsnip['pmax']) )[1:] # skip nan

In [ ]:
cm = confusion_matrix(df_parcut['model_parsnipconf'][iMatrix], df_parcut['pmax'][iMatrix], labels=pmodels)


In [ ]:
norm = cm.sum(axis=1)

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix=cm/np.tile(norm, len(pmodels)).reshape([len(pmodels),len(pmodels)]).transpose(), display_labels=pmodels)

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_axes([0,0,1,1])
disp.plot(ax=ax)
#ax.show()
fig.show()


In [ ]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_axes([0,0,1,1])
disp.plot(ax=ax)
#ax.show()
fig.show()

In [ ]:
# First plot, plot the different classifications for some subset with a min dof
df_parsnip_dof = df_parsnipm2[df_parsnipm2['dof']>5]
df_pardof = df_parsnip_dof.drop(['dof', 'stock'], axis=1)

In [ ]:
for model, group in df_parsnip_dof.groupby(['model']):
    run_ok = sum(group['dof']>0)
    print('Model {} for {} fits out of which {} worked.'.format(model, group.shape[0], run_ok))
#    sub = group[ ~(group['xgbsuccess']==False) ]
#    print('Out of the remaining {}, {} are classified as class O ({})'.format(
#                        sub.shape[0], sum(group['is_0']==True), sum(group['is_0']==True)/sub.shape[0]))


In [ ]:
parnsip_classmean = df_pardof.groupby(['model']).mean()

In [ ]:
parnsip_classstd = df_pardof.groupby(['model']).std()

In [ ]:
parnsip_classmean

In [ ]:
fig, ax = plt.subplots()
plot = parnsip_classmean.loc['agn'].plot(kind='bar',yerr=parnsip_classstd.loc['agn'],colormap='OrRd_r',edgecolor='black',grid=False,figsize=(8,2),ax=ax,position=0.45,error_kw=dict(ecolor='black',elinewidth=0.5),width=0.8)

Next kind of plot, for a given run model (e.g. agn), plot how the fractions of classifications evolve with dof.


In [ ]:
run_model = 'sniasalt2'
comp_classes = ['PISN', 'SLSN-I', 'SNII', 'SNIa', 'SNIa91bg', 'SNIax', 'SNibc']
doflimits = list( range(0,85,5) )

In [ ]:
def get_classevo(df_model, compclass, doflimits):
    """
    Get the classification for entries within dof bins
    """
    means, binds, binnbr = binned_statistic(df_model['dof'], df_model[compclass], bins=doflimits)
    return means
    

In [ ]:
plt.plot(doflimits[1:], means)

In [ ]:
print(df_salt_ampelz.columns)
print(df_z.columns)

In [ ]:
sum(df_salt_ampelz.duplicated('stock'))

We now want to merge the ampelz list with the salt output. The first should be larger than the second.

In [ ]:
df_ampelz = df_z.merge(df_salt_ampelz, how='outer', on='stock', suffixes=('_ampelz',None))

In [ ]:
df_ampelz[ df_ampelz.duplicated('stock') ]

3. We now redo this for results retrieved from BTS. Here we do not need DigestRedshift results. 

In [ ]:
salt_btsz_plots = get_saved_saltfits(7231927385063365296, 4)
salt_btsz_plots = pd.DataFrame.from_dict(salt_btsz_plots)
salt_btsz_plots.shape

In [ ]:
salt_btsz_noplots = get_saved_saltfits(-5764621775949025619, 4)
salt_btsz_noplots = pd.DataFrame.from_dict(salt_btsz_noplots)
salt_btsz_noplots.shape

In [ ]:
salt_btsz = salt_btsz_noplots.append( salt_btsz_plots )
salt_btsz.shape

In [ ]:
# Next step will beto go through the above and for events where muliple rows exist, show "the best"
# to retain. Or, actually we do not need to? We can plot all events and the selection will
# naturally be part of how we cut things down. Will only have to think of how we combine the tables
# and how we do the accounting after cutting.
len(salt_btsz['stock'].unique())

In [ ]:
btsz_unique = []
for stock in salt_btsz['stock'].unique():
    stock_sub = salt_btsz[salt_btsz['stock']==stock]
    
    if stock_sub.shape[0]==1:
        btsz_unique.append( dict(stock_sub.iloc[0]) )
        continue

               
    # As a first step, we can choose the subset with most dof around peak
    stock_sub = stock_sub[ stock_sub['nbr_peak_pulls']==stock_sub['nbr_peak_pulls'].max() ]
    if stock_sub.shape[0]==1:
        btsz_unique.append( dict(stock_sub.iloc[0]) )
        continue
        
    # If some have very few ndof we can use this
    if stock_sub['ndof'].max()>15 and stock_sub['ndof'].min()<15:
        stock_sub = stock_sub[ stock_sub['ndof']>15 ]
    
    
    # We now take the smallest chisq / ndof
    chisqdof = stock_sub['chisq'] / stock_sub['ndof']
    stock_sub = stock_sub[chisqdof==chisqdof.min()]

    # Just test this
    btsz_unique.append( dict(stock_sub.iloc[0]) )


In [ ]:
len(btsz_unique)

In [ ]:
df_salt_btsz = pd.DataFrame.from_dict(btsz_unique)

In [ ]:
df_salt_btsz.shape

In [ ]:
# Done already?
df_btsz = df_salt_btsz

In [ ]:
sum(df_btsz.duplicated('stock'))

3. Next we will look for host information from the Zou et al LS catalog, if available.

In [ ]:
hostinfo = []

In [ ]:
for t2info in col.find({"unit":"T2CatalogMatch", "body.LSPhotoZZou.dist2transient":{"$exists": True} }):
    b = t2info['body'][-1]['LSPhotoZZou']
    store = { 'Zou_'+k:float(v) for k, v in b.items() if k in 
             ['dist2transient','photoz','e_photoz','_6','logMassBest','logMassInf','logMassSup'] }
    store['stock'] = t2info['stock']
    hostinfo.append(store)
    if len(hostinfo)%1000==0:
        print(len(hostinfo))

In [ ]:
df_host = pd.DataFrame.from_dict(hostinfo)

In [ ]:
print(df_host.shape)
df_host = df_host.drop_duplicates()
print(df_host.shape)

In [ ]:
# Again, differences are small, can take the last
for k, stock in enumerate( df_host['stock'][df_host.duplicated('stock')].unique() ):
    df_subset = df_host[df_host['stock']==stock]
    print(df_subset)
    if k>10:
        break

In [ ]:
df_host = df_host.drop_duplicates('stock', keep='last')
print(df_host.shape)

In [ ]:
sum(df_host.duplicated('stock'))

In [ ]:
df_ampelz = df_ampelz.merge(df_host, how='outer', on='stock', suffixes=(None,None))
df_btsz = df_btsz.merge(df_host, how='left', on='stock', suffixes=(None,None))  # No point in adding all

In [ ]:
sum(df_ampelz.duplicated('stock'))

In [ ]:
sum(df_btsz.duplicated('stock'))

2. Extract BTS information if available.

In [ ]:
btsinfo = []

In [ ]:
for t2info in col.find({"body.bts_peakmag":{"$exists": True} }):
    b = t2info['body'][-1]
    store = { k:v for k, v in b.items() if k in 
             ['bts_peakfilt','bts_duration','bts_type'] }
    store['stock'] = t2info['stock']
    
    for p in ['bts_peakt', 'bts_peakmag', 'bts_redshift']:
        try:
            store[p] = float(b[p])
        except ValueError:
            continue
    
    btsinfo.append(store)
    if len(btsinfo)%1000==0:
        print(len(btsinfo))


In [ ]:
df_bts = pd.DataFrame.from_dict(btsinfo)

In [ ]:
df_bts = df_bts.drop_duplicates()

In [ ]:
_ = plt.hist(df_bts['bts_redshift'],bins=100)
#plt.xlim([0,0.3])

In [ ]:
df_bts['bts_type'].unique()

As target BTS type we will consider ['SN Ia', 'SN Ia-91T'], as transient other ['SN IIb', 'SN II', 'SN Ib', 'SN IIP', SN Ic-BL', 'SN Ia-CSM', 'SN Ia-pec', 'SLSN-I', 'SN IIn', 'SN Ic', 'SN Ia-CSM', 'SN Ib/c', SLSN-II', 'SN Iax', 'SN II-pec', 'TDE', 'CV', 'SN Ibn', 'SN Ib-pec', 'SN Ia-91bg', 'SN Ia-SC', 'SN Icsn'] and static ['AGN', 'LBV', 'ILRT']. 

In [ ]:
sum( df_bts['bts_type'].isin(['SN Ia', 'SN Ia-91T']) )

In [ ]:
# Construct a new entry, isIa, which is set to 1 if this is known SNIa, as -1 if known other transient and -2 if other types.
df_bts['isIa'] = 0

In [ ]:
df_bts.loc[ list(df_bts['bts_type'].isin(['SN Ia', 'SN Ia-91T'])), 'isIa' ] = 1

In [ ]:
df_bts.loc[ list(df_bts['bts_type'].isin(['SN IIb', 'SN II', 'SN Ib', 'SN IIP', 'SN Ic-BL', 'SN Ia-CSM', 'SN Ia-pec', 'SLSN-I', 'SN IIn', 'SN Ic', 'SN Ia-CSM', 'SN Ib/c', 'SLSN-II', 'SN Iax', 'SN II-pec', 'TDE', 'CV', 'SN Ibn', 'SN Ib-pec', 'SN Ia-91bg', 'SN Ia-SC', 'SN Icsn'])), 'isIa' ] = -1

In [ ]:
df_bts.loc[ list(df_bts['bts_type'].isin(['AGN', 'LBV', 'ILRT'])), 'isIa' ] = -2

In [ ]:
df_bts['isIa'].hist()

In [ ]:
print('We find %s bts Z entries from %s unique transients.'%(len(df_bts['stock']),len(df_bts['stock'].unique())))

In [ ]:
# Now let us try to merge these were possible

In [ ]:
df_ampelz = df_ampelz.merge(df_bts, how='outer', on='stock', suffixes=(None,None)) # This is where some duplicates enter?
df_btsz = df_btsz.merge(df_bts, how='outer', on='stock', suffixes=(None,None))  # No point in adding all

In [ ]:
df_ampelz['z_dist'].hist()
df_ampelz['z'].hist()

In [ ]:
from astropy.cosmology import Planck18
import numpy as np
from astropy import units as U

In [ ]:
df_ampelz['ang_dist_kpc'] = Planck18.angular_diameter_distance(df_ampelz['z']).values / U.Mpc / ( 360 * 60 * 60 / (2*np.pi) ) * df_ampelz['z_dist'] * 1000

In [ ]:
Planck18.angular_diameter_distance(0.08) / ( 360 * 60 * 60 / (2*np.pi) ) / U.Mpc

In [ ]:
# For bts we will use the Zou data, since we did not add the DigestRedshift stuff. Probably should have?
df_btsz['ang_dist_kpc'] = Planck18.angular_diameter_distance(df_btsz['z']).values / U.Mpc / ( 360 * 60 * 60 / (2*np.pi) ) * df_btsz['Zou_dist2transient'] * 1000

In [ ]:
plt.hist(df_btsz['ang_dist_kpc'],bins=30)

In [ ]:
df_ampelz[ df_ampelz['stock']==304843764 ]

Let us have a look at parsnip. Parsnip was only run based on the ampelz data, but we can in principle add them also to the bts sample.

In [ ]:
pn = get_saved_parsnip(1079222371588722561, 4 )

In [ ]:
df_parsnip = pd.DataFrame.from_dict(pn)
print(df_parsnip.shape)

In [ ]:
# Remove complete duplicates
df_parsnip = df_parsnip.drop_duplicates()
print(df_parsnip.shape)

In [ ]:
# Now we have to look at duplicate states
parsnip_unique = []
for k, stock in enumerate(df_parsnip['stock'].unique()):
    stock_sub = df_parsnip[df_parsnip['stock']==stock]
    
    if stock_sub.shape[0]==1:
        parsnip_unique.append( dict(stock_sub.iloc[0]) )
        continue

    # If some have very few ndof we can remove these
    if stock_sub['model_dof'].max()>15 and stock_sub['model_dof'].min()<15:
        stock_sub = stock_sub[ stock_sub['model_dof']>15 ]

    
    # We now take the smallest chisq / ndof
    try:
        stock_sub = stock_sub[ stock_sub['chi2pdf']==stock_sub['chi2pdf'].min() ]
    except KeyError:
        print(stock_sub)
        raise
        break

    # Just test this
    parsnip_unique.append( dict(stock_sub.iloc[0]) )


In [ ]:
df_parsnip = pd.DataFrame.from_dict(parsnip_unique)
print(df_parsnip.shape)

In [ ]:
sum( df_parsnip.duplicated('stock') )

In [ ]:
df_ampelz = df_ampelz.merge(df_parsnip, how='outer', on='stock', suffixes=(None,'pn'))
df_btsz = df_btsz.merge(df_parsnip, how='left', on='stock', suffixes=(None,'pn'))  # No point in adding all

In [ ]:
df_ampelz[ df_ampelz['stock']==304843764 ]

In [ ]:
sum( df_ampelz.duplicated('stock') )

Lets make an intermession and look for matches to the old pan void catalog

In [ ]:
# initialize the CatalogQuery object pointing it to an existsing database
mqc_query = CatalogQuery.CatalogQuery(
        cat_name = 'voidGalPan',           # name of the database
        coll_name = 'srcs',               # name of the collection with the sources
        ra_key = 'ra', dec_key = 'dec',   # name of catalog fields for the coordinates
        dbclient = None)

In [ ]:
ppcol = db.t0

In [ ]:
df_ampelz['z_void'] = 0

In [ ]:
for i, doc in df_ampelz.iterrows():
    pp = ppcol.find_one({'stock':doc['stock'], 'id':{"$gte":0}})
    ra = pp['body']['ra']
    dec = pp['body']['dec']
    hpcp, hpcp_dist = mqc_query.findclosest(ra, dec, 10, method = 'healpix')
    
    if hpcp is None:
        continue

    print(i)
    print(doc['stock'])
    print(hpcp)
    print(hpcp_dist)
    df_ampelz['z_void'].iloc[i] = hpcp['z']

In [ ]:
df_ampelz[df_ampelz['z_void']>0]

In [ ]:
# Finally, let us save for temporary inspection
df_ampelz.to_csv('/home/jnordin/tmp/allIa_ampelz.csv')
#

In [ ]:
df_btsz.to_csv('/home/jnordin/tmp/allIa_btsz.csv')

In [ ]:
# Which configurations correspond to what setting:
# 3077100070068507562    - AmpelZ with max group 7
# -7755784680569905391   - AmpelZ with max group 3. This can be IGNORED as its a subset of above.
# 7231927385063365296    - BTS z
# 6841134267660678928    - Seems to be the same as 3077100070068507562 but that this does not plot
# -5764621775949025619   - Same as 7231927385063365296 but without plots
# Also add Parsnip while we are here:
# 1079222371588722561    - Parsnip

In [ ]:
def get_saved_saltfits(config, min_dof):
    """
    Go through all sncosmo files with this config, a min degree of freedom and a converged fit.
    
    Return fit metrics and results.
    """
    
    stored = []
    
    for t2info in col.find({"unit":"T2RunSncosmo", "config": config, 
                            "body.sncosmo_result.ndof":{ "$gte": min_dof},
                            "body.sncosmo_result.success": True}):
        bodies = [b for b in t2info['body'] if (
                    'sncosmo_result' in b.keys() 
                    and b['sncosmo_result']['success']
                    and 'paramdict' in b['sncosmo_result'].keys()
                    and b['sncosmo_result']['paramdict']['mwr_v']==3.1 )
               ]
        if len(bodies)==0:
            # Worked with free mw, not w/o? poor lc
            print('no success with fix mw?')
            continue
        body = bodies[-1]
        if not body['sncosmo_result']['success']:
            
            print('success then failure?')
            print(t2info)
            print(body)
            sys.exit('fix!')
        tostore = { k:float(v) for k, v in body['sncosmo_result'].items() if k in 
                 ['chisq','ndof'] }
        tostore['stock'] = t2info['stock']
        tostore.update( body['sncosmo_result']['paramdict'])
        tostore.update( {'{}_err'.format(k):v for k,v in body['sncosmo_result']['errors'].items() } )
        tostore.update( body['fit_metrics'])

        
    
        stored.append(tostore)
        if len(stored)%10000==0:
            print(len(stored))
            
    return stored


In [ ]:
def get_saved_parsnip(config, min_dof):
    """
    Go through all parsnip files with this config, a min degree of freedom and a converged fit.
    
    Return fit metrics and results.
    """
    
    # What to store
    stored = []
    
    for t2info in col.find({"unit":"T2RunParsnip", "config": config, 
                            "body.prediction.model_dof":{ "$gte": min_dof}}):
        body = t2info['body'][-1] 
        tostore = { k:float(v) for k, v in body['prediction'].items() if k not in 
                 ['ra','dec', 'type', ] }
        tostore['stock'] = t2info['stock']
        tostore.update( body['classification'] )
    
        stored.append(tostore)
        if len(stored)%10000==0:
            print(len(stored))
            
    return stored

In [ ]:
is1 = 0.3
is2 = None

In [ ]:
if is2 is None:
    print('foo')

In [ ]:
import re

In [ ]:
sub = 'ztf'

In [ ]:
all = 'baldsflkjasfaua/ztf'

In [ ]:
if re.search(sub, all ):
    print('finding')